<a href="https://colab.research.google.com/github/seokhee516/ML-DL-playground/blob/main/ai08_sc42x_%EC%A0%95%EC%84%9D%ED%9D%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SC42x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N421**
- Stopwords(불용어) : 분석을 하는 것에 있어서는 큰 도움이 되지 않는 단어들을 말합니다.
- Stemming과 Lemmatization : Stemming이란 단어에서 어간과 접사를 분리하는 과정을 의미하고, Lemmatization이란 기본 사전형 단어 형태로 변환하는 것을 의미합니다.
- Bag-of-Words : 단어의 빈도 수를 세어 벡터화하는 것을 말합니다.
- TF-IDF : 특정 문서에만 등장하는 단어에 가중치를 주는 방식입니다.

**N422**
- Word2Vec : 단어를 벡터로(Word to Vector) 나타내는 방법입니다.
- fastText : Word2Vec 방식에 철자(Character) 기반의 임베딩 방식을 더해준 새로운 임베딩 방식입니다.

**N423**
- RNN : 연속형 데이터를 처리하기 위해 고안된 순환 신경망입니다. 
- LSTM, GRU : RNN에 기울기 정보 크기를 조절하기 위한 Gate를 추가한 모델을 LSTM이라고 하고, LSTM의 간소한 버전인 GRU입니다.
- Attention : Attention이란 인코더에 입력된 문장의 단어와 디코더가 생성하려는 단어가 연관된 정도를 나타내는 가중치입니다.

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [3]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [ ]:
!pip install gcsfs
GCS_DS_PATH = "gs://kds-e71c1b9b5ac29135e88e262af6c66b69ee7189c49c9966ff42373099"

In [4]:
import pandas as pd

True_df = pd.read_csv(GCS_DS_PATH+'/True.csv')
Fake_df = pd.read_csv(GCS_DS_PATH+'/Fake.csv')

In [5]:
True_df.shape, Fake_df.shape

((21417, 4), (23481, 4))

In [6]:
True_df['label'] = 0
Fake_df['label'] = 1

In [7]:
df = pd.concat([True_df[['text','label']],Fake_df[['text', 'label']]])

In [8]:
df

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,0
1,WASHINGTON (Reuters) - Transgender people will...,0
2,WASHINGTON (Reuters) - The special counsel inv...,0
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0
...,...,...
23476,21st Century Wire says As 21WIRE reported earl...,1
23477,21st Century Wire says It s a familiar theme. ...,1
23478,Patrick Henningsen 21st Century WireRemember ...,1
23479,21st Century Wire says Al Jazeera America will...,1


## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [9]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=15)
dtm_tfidf = tfidf.fit_transform(df['text'])
dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names())
dtm_tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,clinton,donald,government,house,new,obama,people,president,republican,reuters,said,state,states,trump,year
0,0.000000,0.071750,0.171626,0.515884,0.000000,0.000000,0.140037,0.124764,0.536731,0.065566,0.398552,0.000000,0.084113,0.453636,0.000000
1,0.000000,0.071524,0.085543,0.000000,0.000000,0.378701,0.139597,0.310928,0.089174,0.065360,0.546287,0.000000,0.000000,0.646014,0.000000
2,0.099359,0.000000,0.000000,0.160666,0.145057,0.000000,0.000000,0.116568,0.250738,0.061259,0.372373,0.000000,0.000000,0.847677,0.076444
3,0.133781,0.090260,0.000000,0.324489,0.292965,0.000000,0.088083,0.156952,0.112534,0.164962,0.188016,0.000000,0.000000,0.815243,0.102927
4,0.000000,0.081392,0.292037,0.097536,0.000000,0.000000,0.000000,0.353827,0.000000,0.074377,0.395600,0.000000,0.190833,0.514601,0.556885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0.000000,0.000000,0.000000,0.000000,0.194102,0.712426,0.000000,0.311962,0.000000,0.000000,0.124569,0.406204,0.420635,0.000000,0.000000
44894,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
44895,0.000000,0.000000,0.494734,0.000000,0.522133,0.136887,0.235476,0.059941,0.042978,0.000000,0.000000,0.585366,0.242465,0.000000,0.039309
44896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.704537,0.000000,0.000000,0.709667


### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [15]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_tfidf)
nn.kneighbors([dtm_tfidf.iloc[42]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


(array([[0.        , 0.26300714, 0.26410604, 0.26441934, 0.27398822]]),
 array([[  42, 5766, 6618,    1, 1132]]))

## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [7]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size = 0.8, random_state=42)
(train.shape, test.shape)

((35918, 2), (8980, 2))

In [8]:
X_train = train['text']
y_train = train['label']
X_test = test['text']
y_test = test['label']

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N422에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [9]:
# 이 곳에 답안을 작성하시길 바랍니다
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [18]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, X_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, X_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 2466
테스트용 리뷰의 평균 길이: 2480


In [11]:
max_len = 3000
X_train = pad_sequences(X_train_encoded, maxlen=max_len)
X_test = pad_sequences(X_test_encoded, maxlen=max_len)

In [12]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 32

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=8, validation_split=0.2)

Epoch 1/8
449/449 [==============================] - 35s 76ms/step - loss: 0.6715 - acc: 0.5838 - val_loss: 0.6315 - val_acc: 0.8771
Epoch 2/8
449/449 [==============================] - 30s 67ms/step - loss: 0.5606 - acc: 0.8751 - val_loss: 0.4824 - val_acc: 0.9059
Epoch 3/8
449/449 [==============================] - 30s 68ms/step - loss: 0.4232 - acc: 0.9212 - val_loss: 0.3627 - val_acc: 0.9319
Epoch 4/8
449/449 [==============================] - 30s 67ms/step - loss: 0.3296 - acc: 0.9350 - val_loss: 0.2895 - val_acc: 0.9626
Epoch 5/8
449/449 [==============================] - 30s 66ms/step - loss: 0.2700 - acc: 0.9484 - val_loss: 0.2416 - val_acc: 0.9656
Epoch 6/8
449/449 [==============================] - 30s 66ms/step - loss: 0.2297 - acc: 0.9555 - val_loss: 0.2080 - val_acc: 0.9690
Epoch 7/8
449/449 [==============================] - 30s 68ms/step - loss: 0.1999 - acc: 0.9627 - val_loss: 0.1829 - val_acc: 0.9713
Epoch 8/8
449/449 [==============================] - 30s 66ms/step - 

In [24]:
model.evaluate(X_test, y_test)

281/281 [==============================] - 2s 6ms/step - loss: 0.1633 - acc: 0.9748


[0.1632927507162094, 0.9748329520225525]

### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N423에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [14]:
# 이 곳에 답안을 작성하시길 바랍니다
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

Epoch 1/2
249/449 [===============>..............] - ETA: 45:10 - loss: 0.2762 - acc: 0.9196

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_test, y_test)

### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

*이곳에 답안을 입력해주세요*

각 문장의 길이가 서로 같을 때 기계는 하나의 행렬로 보고, 한꺼번에 병렬 연산 처리 할 수 있다. pad_sequences를 통해 이러한 문장의 길이를 맞춰주는 작업을 할 수 있다. 

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

*이곳에 답안을 입력해주세요* 

- 단어 벡터의 평균을 이용한 분류(0.1632927507162094, 0.9748329520225525)
- LSTM을 이용한 분류 (램 사용량 초과로 세션 다운됐습니다.)
- 단어 벡터의 평균을 이용한 분류는 빠르다는 장점이 있고, LSTM을 이용한 분류는 연산 속도가 느리다는 단점이 있다.

#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

*이곳에 답안을 입력해주세요*

RNN에 비해 LSTM은 기울기 폭발, 기울기 소실과 장기 의존성 문제를 개선했다.



#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

*이곳에 답안을 입력해주세요*

이미지 캡셔닝, 기계번역, 비디오 프레임별 분류

연속된 데이터를 처리하기 위해 고안된 신경망이기 때문이다.

#### e) 이외에 N424 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

*이곳에 답안을 입력해주세요*

Transformer, Self-Attention, 인코더, 디코더

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [ ]:
# 이 곳에 답안을 작성하시길 바랍니다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vect = CountVectorizer(stop_words='english', max_features=100)
dtm_count= count_vect.fit_transform(df['text'])

cosine_sim = cosine_similarity(dtm_count, dtm_count)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

CountVectorizer는 TF-IDF와 달리 단순히 단어들의 빈도만 고려하여 벡터화합니다.